In [2]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import normalize, LabelEncoder
from patsy import dmatrix
from sklearn.linear_model import LinearRegression, LassoCV, LassoLarsCV, RidgeCV, ElasticNetCV, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn import cross_validation
from sklearn.metrics import explained_variance_score, mean_squared_error, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import xgboost as xgb
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [31]:
df = pd.read_csv('../dataset/mlb_fa_list2.csv')

print(df.shape)
df.tail()

(1225, 72)


,Name,Yr,Team,G,AB,PA,H_b,1B,2B,3B,HR_b,R_b,RBI,BB_b,IBB_b,SO_b,BB%,K%,BB/K,BABIP_b,HBP_b,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC+,WPA,W,L,GS,CG,IP,H_p,HR_p,BB_p,SO_p,R_p,SV,BS,BK,ER,HBP_p,HLD,IBB_p,ShO,TBF,WP,HR/9,BB/9,K/9,K/BB,K-BB%,GB%,HR/FB,LOB%,ERA,BABIP_p,FIP,WHIP,Contract Yrs,Total Salary($),Avg Salary($)
1220,Trevor Plouffe,2016,Twins,84,319.0,344.0,83.0,57.0,13.0,1.0,12.0,35.0,47.0,19.0,0.0,60.0,5.5,17.4,0.32,0.284,2.0,3.0,1.0,11.0,1.0,0.0,-8.4,0.160,0.311,-2.1,0.260,0.303,0.420,0.723,-0.4,91.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,1.0,5250000.0,5250000.0
1221,Tyson Ross,2016,Padres,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.1,0.0,-0.24,0.0,1.0,1.0,0.0,5.1,9.0,0.0,1.0,5.0,8.0,0.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0,27.0,1.0,0.0,1.69,8.44,5.0,14.8,47.4,0.0,33.3,11.81,0.474,2.96,1.88,1.0,6000000.0,6000000.0
1222,Welington Castillo,2016,Diamondbacks,113,416.0,457.0,110.0,72.0,24.0,0.0,14.0,41.0,68.0,33.0,3.0,121.0,7.2,26.5,0.27,0.337,4.0,4.0,0.0,5.0,2.0,0.0,9.8,0.159,0.319,0.4,0.264,0.322,0.423,0.745,1.6,91.0,0.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,2.0,13000000.0,6500000.0
1223,Wilson Ramos,2016,Nationals,131,482.0,523.0,148.0,101.0,25.0,0.0,22.0,58.0,80.0,35.0,2.0,79.0,6.7,15.1,0.44,0.327,2.0,4.0,0.0,17.0,0.0,0.0,8.4,0.189,0.361,18.7,0.307,0.354,0.496,0.850,3.4,124.0,2.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,2.0,12500000.0,6250000.0
1224,Yoenis Cespedes,2016,Mets,132,479.0,543.0,134.0,77.0,25.0,1.0,31.0,72.0,86.0,51.0,8.0,108.0,9.4,19.9,0.47,0.298,7.0,6.0,0.0,14.0,3.0,1.0,-9.2,0.251,0.369,22.8,0.280,0.354,0.530,0.884,3.2,135.0,3.21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,4.0,110000000.0,27500000.0


In [94]:
pd.options.display.max_columns = 300

In [164]:
df2 = pd.get_dummies(df.iloc[:, 1:], columns=['Team'])

In [165]:
idx_2016 = 1136

In [166]:
np.vectorize(max)(0.0, df['BABIP_p'])

array([ 0.   ,  0.207,  0.311, ...,  0.   ,  0.   ,  0.   ])

In [167]:
pd.Series(np.log1p(np.vectorize(max)(0, df['BABIP_p']))).value_counts()

0.0    1225
dtype: int64

In [168]:
df2.rename(columns={"wRC+": "wRC_Plus", "Contract Yrs" : "Contract_Yrs"}, inplace=True)
df2.rename(columns={"BB%": "BB_Percent", "K%": "K_Percent", "K-BB%": "K_BB_Percent", "GB%": "GB_Percent",
                   "LOB%": "LOB_Percent"}, inplace=True)
df2.rename(columns={"Avg Salary($)": "Salary", "Total Salary($)" : "Total_Salary"}, inplace=True)
df2.rename(columns={"1B" : "Hits", "2B" : "Doubles", "3B" : "Triples"}, inplace=True)
df2.rename(columns={"BB/K" : "BB_per_K", "HR/9" : "HR_per_9", "BB/9" : "BB_per_9", "K/9" : "K_per_9", "K/BB" : "K_per_BB",
                   "HR/FB" : "HR_per_FB"}, inplace=True)

df2.tail()

,Yr,G,AB,PA,H_b,Hits,Doubles,Triples,HR_b,R_b,RBI,BB_b,IBB_b,SO_b,BB_Percent,K_Percent,BB_per_K,BABIP_b,HBP_b,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC_Plus,WPA,W,L,GS,CG,IP,H_p,HR_p,BB_p,SO_p,R_p,SV,BS,BK,ER,HBP_p,HLD,IBB_p,ShO,TBF,WP,HR_per_9,BB_per_9,K_per_9,K_per_BB,K_BB_Percent,GB_Percent,HR_per_FB,LOB_Percent,ERA,BABIP_p,FIP,WHIP,Contract_Yrs,Total_Salary,Salary,Team_- - -,Team_Angels,Team_Astros,Team_Athletics,Team_Blue Jays,Team_Braves,Team_Brewers,Team_Cardinals,Team_Cubs,Team_Devil Rays,Team_Diamondbacks,Team_Dodgers,Team_Giants,Team_Indians,Team_Mariners,Team_Marlins,Team_Mets,Team_Nationals,Team_Orioles,Team_Padres,Team_Phillies,Team_Pirates,Team_Rangers,Team_Rays,Team_Red Sox,Team_Reds,Team_Rockies,Team_Royals,Team_Tigers,Team_Twins,Team_White Sox,Team_Yankees
1220,2016,84,319.0,344.0,83.0,57.0,13.0,1.0,12.0,35.0,47.0,19.0,0.0,60.0,5.5,17.4,0.32,0.284,2.0,3.0,1.0,11.0,1.0,0.0,-8.4,0.160,0.311,-2.1,0.260,0.303,0.420,0.723,-0.4,91.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,1.0,5250000.0,5250000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1221,2016,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.1,0.0,-0.24,0.0,1.0,1.0,0.0,5.1,9.0,0.0,1.0,5.0,8.0,0.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0,27.0,1.0,0.0,1.69,8.44,5.0,14.8,47.4,0.0,33.3,11.81,0.474,2.96,1.88,1.0,6000000.0,6000000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1222,2016,113,416.0,457.0,110.0,72.0,24.0,0.0,14.0,41.0,68.0,33.0,3.0,121.0,7.2,26.5,0.27,0.337,4.0,4.0,0.0,5.0,2.0,0.0,9.8,0.159,0.319,0.4,0.264,0.322,0.423,0.745,1.6,91.0,0.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,2.0,13000000.0,6500000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1223,2016,131,482.0,523.0,148.0,101.0,25.0,0.0,22.0,58.0,80.0,35.0,2.0,79.0,6.7,15.1,0.44,0.327,2.0,4.0,0.0,17.0,0.0,0.0,8.4,0.189,0.361,18.7,0.307,0.354,0.496,0.850,3.4,124.0,2.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,2.0,12500000.0,6250000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1224,2016,132,479.0,543.0,134.0,77.0,25.0,1.0,31.0,72.0,86.0,51.0,8.0,108.0,9.4,19.9,0.47,0.298,7.0,6.0,0.0,14.0,3.0,1.0,-9.2,0.251,0.369,22.8,0.280,0.354,0.530,0.884,3.2,135.0,3.21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.00,0.00,4.0,110000000.0,27500000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [169]:
def scaled(col):
    return (col - col[:idx_2016].mean()) / (col[:idx_2016].std())

In [170]:
np.vectorize(max)(df2['WAR'], 0)[:20]

array([ 0.2,  0.6,  1.7,  3.2,  0. ,  0.7,  2.1,  2.4,  5.8,  2.9,  0.2,
        3. ,  1.7,  1.8,  0.3,  1.9,  0. ,  1.5,  0.6,  3.7])

In [171]:
for col in df2.columns:
    if not("Yr_" in col or "Team_" in col):
        df2[col] = np.log1p(np.vectorize(max)(0.0, df2[col]))
        if col != "Salary":
            df2[col] = scaled(df2[col])

In [172]:
df2.tail()

,Yr,G,AB,PA,H_b,Hits,Doubles,Triples,HR_b,R_b,RBI,BB_b,IBB_b,SO_b,BB_Percent,K_Percent,BB_per_K,BABIP_b,HBP_b,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC_Plus,WPA,W,L,GS,CG,IP,H_p,HR_p,BB_p,SO_p,R_p,SV,BS,BK,ER,HBP_p,HLD,IBB_p,ShO,TBF,WP,HR_per_9,BB_per_9,K_per_9,K_per_BB,K_BB_Percent,GB_Percent,HR_per_FB,LOB_Percent,ERA,BABIP_p,FIP,WHIP,Contract_Yrs,Total_Salary,Salary,Team_- - -,Team_Angels,Team_Astros,Team_Athletics,Team_Blue Jays,Team_Braves,Team_Brewers,Team_Cardinals,Team_Cubs,Team_Devil Rays,Team_Diamondbacks,Team_Dodgers,Team_Giants,Team_Indians,Team_Mariners,Team_Marlins,Team_Mets,Team_Nationals,Team_Orioles,Team_Padres,Team_Phillies,Team_Pirates,Team_Rangers,Team_Rays,Team_Red Sox,Team_Reds,Team_Rockies,Team_Royals,Team_Tigers,Team_Twins,Team_White Sox,Team_Yankees
1220,1.809166,0.383841,0.920287,0.906930,0.929448,0.942594,0.774313,0.512201,1.145101,0.853713,1.029225,0.686152,-0.701334,0.904022,0.630584,0.903959,0.278394,0.867227,0.488871,0.979380,0.385230,1.242873,0.025677,-0.678786,-0.530176,1.003697,0.888137,-0.506772,0.951864,0.812493,0.997913,0.923829,-1.125622,0.938482,0.199940,-0.80009,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,-0.608809,0.179039,15.473739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1221,1.809166,-4.895864,-1.079742,-1.080309,-1.062871,-1.058057,-1.018915,-0.633100,-0.915871,-1.046178,-1.039673,-1.027538,-0.701334,-1.069451,-1.048525,-1.073535,-0.939341,-1.072163,-0.798178,-0.846811,-0.597059,-0.967440,-0.674764,-0.678786,-0.530176,-0.943671,-1.068746,-0.506772,-1.066690,-1.070734,-1.055531,-1.069732,-0.948429,-1.056419,-0.680949,-0.80009,-0.079011,0.035673,-0.263,-0.111914,0.117623,-0.852445,-0.507336,-0.071219,0.269051,-0.373713,-0.506246,-0.324979,0.231516,0.677951,-0.54345,-0.739715,-0.223687,0.201482,0.182659,-0.855044,0.443028,1.182355,1.863015,1.337750,1.091247,-0.916209,0.686295,2.204873,2.034034,0.744984,1.517645,-0.608809,0.275890,15.607270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1222,1.809166,0.797187,1.012089,1.003282,1.054772,1.055927,1.168297,-0.633100,1.260084,0.935440,1.223173,0.989694,1.184759,1.236775,0.838998,1.176806,0.109024,1.181031,1.087315,1.273331,-0.597059,0.626322,0.435408,-0.678786,2.288739,0.992382,0.932101,-0.178657,0.979548,0.915499,1.010272,0.970319,0.650789,0.938482,0.393798,-0.80009,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,0.552353,0.836686,15.687313,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1223,1.809166,1.003621,1.063033,1.049063,1.187156,1.220747,1.194947,-0.633100,1.603541,1.115629,1.308866,1.022391,0.793359,1.034192,0.782561,0.813291,0.660039,1.122786,0.488871,1.273331,-0.597059,1.603532,-0.674764,-0.678786,2.124267,1.327681,1.158623,2.399819,1.271731,1.085681,1.303233,1.184421,1.628860,1.073713,2.089828,-0.80009,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,0.552353,0.808239,15.648092,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1224,1.809166,1.014248,1.060873,1.061802,1.142789,1.088569,1.194947,0.512201,1.868895,1.228515,1.347056,1.232746,2.288053,1.182685,1.052202,0.990463,0.750479,0.951360,1.637936,1.716572,-0.597059,1.441358,0.726118,0.349547,-0.530176,1.994612,1.200977,2.584190,1

In [173]:
df_train = df2.iloc[:idx_2016, :].copy()

In [174]:
df_test = df2.iloc[idx_2016:, :].copy()
df_test.reset_index(inplace=True, drop=True)

In [175]:
df_train.tail()

,Yr,G,AB,PA,H_b,Hits,Doubles,Triples,HR_b,R_b,RBI,BB_b,IBB_b,SO_b,BB_Percent,K_Percent,BB_per_K,BABIP_b,HBP_b,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC_Plus,WPA,W,L,GS,CG,IP,H_p,HR_p,BB_p,SO_p,R_p,SV,BS,BK,ER,HBP_p,HLD,IBB_p,ShO,TBF,WP,HR_per_9,BB_per_9,K_per_9,K_per_BB,K_BB_Percent,GB_Percent,HR_per_FB,LOB_Percent,ERA,BABIP_p,FIP,WHIP,Contract_Yrs,Total_Salary,Salary,Team_- - -,Team_Angels,Team_Astros,Team_Athletics,Team_Blue Jays,Team_Braves,Team_Brewers,Team_Cardinals,Team_Cubs,Team_Devil Rays,Team_Diamondbacks,Team_Dodgers,Team_Giants,Team_Indians,Team_Mariners,Team_Marlins,Team_Mets,Team_Nationals,Team_Orioles,Team_Padres,Team_Phillies,Team_Pirates,Team_Rangers,Team_Rays,Team_Red Sox,Team_Reds,Team_Rockies,Team_Royals,Team_Tigers,Team_Twins,Team_White Sox,Team_Yankees
1131,1.492315,0.784781,0.933052,0.923288,0.907510,0.951017,0.723957,-0.63310,0.934287,0.592615,0.931786,0.740673,-0.701334,1.164737,0.671059,1.231345,-0.139651,1.081753,1.481500,0.066284,0.959832,0.986981,-0.674764,0.349547,1.251620,0.519825,0.760229,-0.506772,0.805069,0.768673,0.727848,0.769639,-0.371813,0.887716,-0.680949,-0.800090,-0.807506,-0.512063,-0.263000,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,0.552353,0.185914,14.790071,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1132,1.492315,0.367177,0.842918,0.824828,0.739029,0.721105,0.394056,1.18216,0.934287,0.638746,0.778039,0.393937,-0.701334,0.904022,0.412993,1.061353,-0.213369,0.577916,-0.798178,1.273331,-0.597059,0.464147,0.435408,0.349547,-0.530176,0.878684,0.595660,-0.506772,0.612633,0.465605,0.749401,0.657025,-1.125622,0.785217,-0.680949,-0.800090,-0.807506,-0.512063,-0.263000,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,-0.608809,-0.891439,13.997833,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1133,1.492315,-0.906395,-1.079742,-1.080309,-1.062871,-1.058057,-1.018915,-0.63310,-0.915871,-1.046178,-1.039673,-1.027538,-0.701334,-1.069451,-1.048525,-1.073535,-0.939341,-1.072163,-0.798178,-0.846811,-0.597059,-0.967440,-0.674764,-0.678786,-0.530176,-0.943671,-1.068746,-0.506772,-1.066690,-1.070734,-1.055531,-1.069732,1.149524,-1.056419,1.579094,1.990825,1.804120,2.274524,-0.263000,1.437353,1.466854,1.464888,1.596044,1.372604,1.425736,-0.373713,-0.506246,-0.324979,1.433042,0.153366,-0.54345,-0.739715,-0.223687,1.350290,2.459610,0.635849,1.095754,0.886807,0.674683,0.757527,1.111547,0.979118,1.069124,0.897450,1.087195,1.031240,1.112749,0.552353,1.218263,16.213406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1134,1.492315,-0.575715,-1.079742,-1.080309,-1.062871,-1.058057,-1.018915,-0.63310,-0.915871,-1.046178,-1.039673,-1.027538,-0.701334,-1.069451,-1.048525,-1.073535,-0.939341,-1.072163,-0.798178,-0.846811,-0.597059,-0.967440,-0.674764,-0.678786,-0.530176,-0.943671,-1.068746,-0.506772,-1.066690,-1.070734,-1.055531,-1.069732,-1.125622,-1.056419,-0.196161,-0.067057,-0.079011,0.035673,-0.263000,1.039168,1.040452,1.224443,0.727882,1.032446,0.969170,0.467089,-0.506246,-0.324979,0.994451,0.153366,-0.54345,0.878439,-0.223687,1.034938,1.108463,1.410473,0.488374,1.024377,1.559609,1.290408,0.898830,1.075042,1.073852,0.965052,0.938440,1.053140,0.869743,-0.608809,-0.226852,14.914123,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1135,1.492315,-0.948431,0.383273,0.401296,0.183825,0.166300,-0.272413,-0.63310,-0.033121,0.118735,-0.298791,-0.631028,-0.701334,0.230580,-0.301360,0.932857,-0.642580,0.659024,-0.798178,0.066284,2.516723,-0.967440,0.025677,-0.678786,2.072753,0.53

In [176]:
df_test.head()

,Yr,G,AB,PA,H_b,Hits,Doubles,Triples,HR_b,R_b,RBI,BB_b,IBB_b,SO_b,BB_Percent,K_Percent,BB_per_K,BABIP_b,HBP_b,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC_Plus,WPA,W,L,GS,CG,IP,H_p,HR_p,BB_p,SO_p,R_p,SV,BS,BK,ER,HBP_p,HLD,IBB_p,ShO,TBF,WP,HR_per_9,BB_per_9,K_per_9,K_per_BB,K_BB_Percent,GB_Percent,HR_per_FB,LOB_Percent,ERA,BABIP_p,FIP,WHIP,Contract_Yrs,Total_Salary,Salary,Team_- - -,Team_Angels,Team_Astros,Team_Athletics,Team_Blue Jays,Team_Braves,Team_Brewers,Team_Cardinals,Team_Cubs,Team_Devil Rays,Team_Diamondbacks,Team_Dodgers,Team_Giants,Team_Indians,Team_Mariners,Team_Marlins,Team_Mets,Team_Nationals,Team_Orioles,Team_Padres,Team_Phillies,Team_Pirates,Team_Rangers,Team_Rays,Team_Red Sox,Team_Reds,Team_Rockies,Team_Royals,Team_Tigers,Team_Twins,Team_White Sox,Team_Yankees
0,1.809166,0.006097,0.705031,0.716373,0.572772,0.583778,0.474089,-0.633100,-0.033121,0.271257,0.636039,0.686152,0.793359,0.594316,1.077713,0.842189,1.149495,0.671427,0.488871,0.066284,1.367519,0.626322,0.435408,0.349547,1.082097,0.090384,0.635728,-0.506772,0.641411,0.801564,0.471856,0.650157,-1.125622,0.785217,-0.680949,-0.80009,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,-0.608809,-0.359090,14.731802,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1.809166,0.949247,0.999387,0.982619,0.994150,0.959298,0.945080,-0.633100,1.530444,1.017167,1.139498,0.857825,1.184759,1.090735,0.697062,1.015968,0.177559,0.714683,0.013860,0.600411,-0.597059,1.441358,-0.674764,0.349547,-0.530176,1.360744,0.849447,-0.506772,0.805069,0.719049,1.043101,0.911047,-1.125622,0.943251,-0.680949,-0.80009,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,-0.608809,-0.729593,14.220976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.809166,0.694734,0.782400,0.796035,0.696172,0.464953,0.723957,1.657503,1.204647,0.882378,0.875478,0.918096,0.793359,1.085371,1.231430,1.369050,0.311497,1.010901,0.013860,1.273331,-0.597059,0.009770,0.951609,-0.678786,1.489343,2.150999,1.046250,1.155636,0.734290,0.899334,1.299317,1.112415,0.916831,1.040747,0.299031,-0.80009,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,-0.608809,-0.861831,14.038655,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1.809166,-0.154349,0.700975,0.738105,0.560780,0.507825,0.303322,-0.633100,0.754989,0.542084,0.288343,1.068179,-0.701334,1.028153,1.602190,1.401735,0.809752,1.204207,0.013860,0.066284,-0.597059,0.265662,-0.674764,-0.678786,1.531657,1.003697,0.986682,0.497275,0.619836,1.111908,0.800808,0.942919,0.253729,1.005117,-0.680949,-0.80009,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.54345,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,-0.608809,-0.520936,14.508658,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1.809166,0.027595,0.636124,0.625939,0.560780,0.693725,-0.272413,-0.633100,-0.915871,0.174594,0.288343,0.229370,0.793359,0.512754,0.602548,0.900259,0.245039,1.040502,-0.798178,0.066284,0.385230,0.626322,1.652050,0.951083,-0.530176,-0.761256,0.532243,-0.506772,0.931044,0.768

In [177]:
df_trainY = df_train['Salary'].copy()
df_testY = df_test['Salary'].copy()

for col in ['Contract_Yrs', 'Total_Salary', 'Salary']:
    try:
        del df_train[col]
        del df_test[col]
    except:
        pass

In [178]:
df_train.head()

,Yr,G,AB,PA,H_b,Hits,Doubles,Triples,HR_b,R_b,RBI,BB_b,IBB_b,SO_b,BB_Percent,K_Percent,BB_per_K,BABIP_b,HBP_b,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC_Plus,WPA,W,L,GS,CG,IP,H_p,HR_p,BB_p,SO_p,R_p,SV,BS,BK,ER,HBP_p,HLD,IBB_p,ShO,TBF,WP,HR_per_9,BB_per_9,K_per_9,K_per_BB,K_BB_Percent,GB_Percent,HR_per_FB,LOB_Percent,ERA,BABIP_p,FIP,WHIP,Team_- - -,Team_Angels,Team_Astros,Team_Athletics,Team_Blue Jays,Team_Braves,Team_Brewers,Team_Cardinals,Team_Cubs,Team_Devil Rays,Team_Diamondbacks,Team_Dodgers,Team_Giants,Team_Indians,Team_Mariners,Team_Marlins,Team_Mets,Team_Nationals,Team_Orioles,Team_Padres,Team_Phillies,Team_Pirates,Team_Rangers,Team_Rays,Team_Red Sox,Team_Reds,Team_Rockies,Team_Royals,Team_Tigers,Team_Twins,Team_White Sox,Team_Yankees
0,-1.68487,1.126142,1.083250,1.075284,1.108183,1.153572,1.016672,0.512201,1.360654,1.141924,1.157314,1.022391,1.184759,1.106476,0.722333,0.862102,0.473366,0.745433,1.899353,1.716572,1.683745,1.552690,1.652050,1.377880,-0.530176,0.717833,0.804976,-0.506772,0.833221,0.790617,0.822094,0.824675,-0.786664,0.882303,-0.680949,-0.800090,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.543450,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.68487,-0.037910,-1.079742,-1.080309,-1.062871,-1.058057,-1.018915,-0.633100,-0.915871,-1.046178,-1.039673,-1.027538,-0.701334,-1.069451,-1.048525,-1.073535,-0.939341,-1.072163,-0.798178,-0.846811,-0.597059,-0.967440,-0.674764,-0.678786,-0.530176,-0.943671,-1.068746,-0.506772,-1.066690,-1.070734,-1.055531,-1.069732,-0.251828,-1.056419,-0.493725,0.901959,-0.807506,-0.512063,-0.263,1.015572,0.831360,0.773949,0.968734,0.990740,0.750685,-0.373713,0.635364,-0.324979,0.707560,1.574735,0.549485,0.878439,-0.223687,1.000792,-0.743144,0.667114,0.940620,0.991812,0.962780,1.102113,0.997300,0.889740,1.091732,0.515864,0.499550,0.973433,0.634070,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,-1.68487,-1.339366,-1.079742,-1.080309,-1.062871,-1.058057,-1.018915,-0.633100,-0.915871,-1.046178,-1.039673,-1.027538,-0.701334,-1.069451,-1.048525,-1.073535,-0.939341,-1.072163,-0.798178,-0.846811,-0.597059,-0.967440,-0.674764,-0.678786,-0.530176,-0.943671,-1.068746,-0.506772,-1.066690,-1.070734,-1.055531,-1.069732,0.720953,-1.056419,-0.680949,1.827804,1.075625,1.965655,-0.263,1.274130,1.321661,1.410947,1.342136,1.282061,1.382022,-0.373713,-0.506246,-0.324979,1.358446,1.574735,-0.543450,2.126431,-0.223687,1.235850,1.650023,1.002983,1.016996,1.024377,0.925444,1.038071,1.013956,1.044282,1.026214,1.113564,1.134194,1.036141,1.122343,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.68487,0.982122,1.012089,1.005502,1.111766,1.215893,1.140558,-0.633100,-0.033121,1.027878,0.904373,0.918096,0.241712,0.934512,0.734707,0.760697,0.690393,1.244599,1.637936,0.979380,1.942121,0.626322,2.352490,1.708929,2.790807,-0.055945,0.953983,-0.506772,1.224827,1.085681,0.787998,0.925955,1.542373,0.970834,0.620047,-0.800090,-0.807506,-0.512063,-0.263,-0.932846,-0.930078,-0.852445,-0.919076,-0.930037,-0.914924,-0.373713,-0.506246,-0.324979,-0.912887,-0.743417,-0.543450,-0.739715,-0.223687,-0.942972,-0.743144,-0.855044,-0.920449,-0.944313,-0.904672,-0.890193,-0.954049,-0.916209,-0.955878,-0.928541,-0.945066,-0.944421,-0.943501,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1.68487,0.069631,-1.079742,-1.080309,-1.062871,-1.058057,-1.018915,-0.633100,-0.915871,-1.046178,-1.039673,-1.027538,-0.701334,-1.069451,-1.048525,-1.073535,-0.939341,-1.072163,-0.798178,-0.846811,-0.597059,-0.967440,-0.674764,-0.678786,-0.530176,-0.943671,-1.068746,-0.506772,-1.066690,-1.070734,-1.055531,-1.069732,-1.125622,-1.056419,-0.680949,0.361739,1.075625,-0.512063,-0.263,0.869599,0.955093,1.151719,0.689

In [179]:
mod = sm.OLS(df_trainY, df_train).fit()
res = mod.summary()

In [180]:
print(res)

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     14.09
Date:                Wed, 24 Jan 2018   Prob (F-statistic):          8.34e-133
Time:                        18:54:22   Log-Likelihood:                -1094.7
No. Observations:                1136   AIC:                             2387.
Df Residuals:                    1037   BIC:                             2886.
Df Model:                          98                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Yr                    0.1015      0.02

In [181]:
np.exp(mod.predict(df_train))

0       2.014350e+06
1       1.340311e+06
2       3.276234e+06
3       2.801756e+06
4       1.465125e+06
5       1.904137e+06
6       2.104700e+06
7       9.306723e+06
8       1.442958e+07
9       8.450111e+06
10      3.882626e+06
11      8.414772e+06
12      3.039968e+06
13      7.069974e+06
14      7.560899e+05
15      3.124073e+06
16      8.771127e+05
17      2.497862e+06
18      6.386292e+05
19      8.669306e+06
20      3.782898e+06
21      2.671506e+06
22      1.366373e+06
23      5.193781e+06
24      3.725679e+06
25      2.530319e+06
26      3.542566e+06
27      2.203824e+06
28      7.385885e+06
29      1.936042e+06
            ...     
1106    1.306745e+07
1107    4.241062e+06
1108    4.868833e+06
1109    2.976813e+06
1110    1.677681e+06
1111    4.614562e+06
1112    4.907066e+06
1113    8.856677e+06
1114    9.296439e+05
1115    1.126718e+06
1116    4.606604e+06
1117    2.675724e+06
1118    2.582784e+06
1119    7.775784e+06
1120    1.069919e+06
1121    1.570914e+06
1122    3.709

In [182]:
df_train0 = df_train.iloc[:1, :]

In [183]:
df_test0 = df_test.iloc[:1, :]

In [184]:
mod.predict(df_train0)

0    14.515807
dtype: float64

In [185]:
mod.predict(df_test0)

0    14.144679
dtype: float64

In [191]:
y_true = np.exp(df_testY)

In [192]:
y_pred = np.exp(mod.predict(df_test))

In [193]:
np.sqrt(mean_squared_error(y_true, y_pred))

3566113.7466542884